In [2]:
# https://www.kaggle.com/bulentsiyah/hepsi-burada-yorum veri seti kullanılmıştır

# Kütüphaneler
import pandas as pd
import numpy as np

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Flatten, Dense, Dropout, GRU, Embedding
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
import time
import warnings

warnings.filterwarnings('ignore')

In [3]:
# Çalışma süresini ölçmek için
start = time. time()

In [4]:
# Veri setinin yüklenmesi
training = pd.read_csv('hepsiburada.csv')

Y = training['Rating'].values.tolist()  # 1:pozitif, 0:negatif
X = training['Review'].values.tolist()  # Cümleler

In [8]:
# Veri setini bölmek
# %25 test, %75 eğitim için
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.25, random_state=2)

In [6]:
num_words = 10000  # Külliyat içinde en sık geçen kaç kelime için tokenleştirme yapılacak
tokenizer = Tokenizer(num_words=num_words) #num_words parametresi verilmezse tüm dataset kullanılır
tokenizer.fit_on_texts(X) #Tokenleştirme yapılır, kelimeler integer'a dönüştürülür, en çok geçen en düşük integer'ı alır
#tokenizer bir sözlüktür, key:kelimeler, value:tokenler


In [9]:
# Eğitim ve test verilerinin tokenleştirilmesi
x_train_tokens = tokenizer.texts_to_sequences(x_train) 
x_test_tokens = tokenizer.texts_to_sequences(x_test)   


In [11]:
# Padding işlemi 
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)        
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens) # ortalama+2*std, 59 çıkacak
max_tokens = int(max_tokens)
np.sum(num_tokens < max_tokens) / len(num_tokens) # Yorumların %kaçı 59 dan az kelime içeriyor=%95.9
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)   #59dan küçükse 0 eklenir, büyükse atılır
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)

In [12]:
# Tokenlerden kelime elde etmek
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

In [13]:
# Eğitim modeli
model = Sequential()  
embedding_size = 50   # Vektör sayısı, burada word2vec yada Glove algoritmalarıda kullanılabilir
model.add(Embedding(input_dim=num_words,output_dim=embedding_size,input_length=max_tokens,name='embedding_layer'))

# Cuda destekli GPU var ise GRU yerine CuDNNGRU kullanarak eğitim hızlandırabilir

model.add(GRU(units=32, return_sequences=True, dropout=0.1)) 
model.add(GRU(units=16, return_sequences=True, dropout=0.1))  #GRU layer 16 output verecek, return_sequence=True ard arda output üretmek için
model.add(GRU(units=8, return_sequences=True, dropout=0.1))   #GRU layer 8 output verecek
model.add(GRU(units=4, dropout=0.1))                          #GRU layer 4 output verecek, return_sequence=False, çünkü bir sonraki layer=output
model.add(Dense(1, activation='sigmoid'))        #Dense layer

optimizer = Adam(lr=1e-3)
model.compile(loss='binary_crossentropy',optimizer=optimizer, metrics=['accuracy']) # İki sınıflı sınıflandırıcı
model.fit(x_train_pad, y_train, epochs=1, batch_size=256) # 16 epoch yapıldığında doğruluk %99 üstüne çıkıyor


Epoch 1/1
182622/182622 [==============================] - 153s 837us/step - loss: 0.1749 - acc: 0.9522


In [16]:
sonuclar = model.evaluate(x_test_pad, y_test) # sonuclar[1] Accuracy 

y_pred = model.predict(x=x_test_pad[0:1000])  #ilk 10000 veri için tahmin et
y_pred = y_pred.T[0]                          #Sütun veriyi vektöre çevir

cls_pred = np.array([1.0 if p>0.5 else 0.0 for p in y_pred])  #0.5 treshold, büyükse 1, küçükse 0
cls_true = np.array(y_test[0:1000])                           #Gerçek sınıflar

incorrect = np.where(cls_pred != cls_true)
incorrect = incorrect[0]

print(len(incorrect))

idx = incorrect[0]    #ilk yanlış tahmin
text = x_test[idx]    #bunun texti
y_pred[idx]           #modelin tahmini
cls_true[idx]         #gerçek sınıf

end = time. time()

print("zaman:", end-start)


60875/60875 [==============================] - 68s 1ms/step
32
zaman: 1350.3059649467468


In [20]:
# Modeli metin veri ile denemek
tst=["ürün kargodan zamanında geldi 6 aydır sorunsuz kullanıyorum", "tek kelime ile berbat hiç beklediğim gibi değil"]
tokens = tokenizer.texts_to_sequences(tst)
tokens_pad = pad_sequences(tokens, maxlen=max_tokens)
tokens_pad.shape
print(model.predict(tokens_pad))

[[0.9840717]
 [0.3818984]]
